In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import cv2


Filtering out unwanted classes.

In [2]:
DATA_PATH = "/kaggle/input/fer2013-dataset/fer2013.csv"
df = pd.read_csv(DATA_PATH)
# Keep only required emotions
selected_classes = [0, 1, 3, 4, 6]   # angry, disgust, happy, sad, neutral

df = df[df['emotion'].isin(selected_classes)]
df = df.reset_index(drop=True)

print("Total samples after filtering:", len(df))


Total samples after filtering: 26764


This class defines a custom PyTorch dataset that reads FER2013 images from a DataFrame, converts them to 3‑channel 224×224 images, optionally applies transforms, and returns them with numeric labels.

In [3]:
class FERDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        pixels = np.array(row['pixels'].split(), dtype="uint8")
        image = pixels.reshape(48, 48)
        
        # Convert grayscale to RGB (ResNet needs 3 channels)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Resize to 224x224 for ResNet requirements
        image = cv2.resize(image, (224, 224))

        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        label = selected_classes.index(row['emotion'])
        return image, label


Test and Train Split

In [4]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['emotion'])

print(len(train_df), len(val_df))


24087 2677


Transformation before feeding it to the model.

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


train,val -> processed img, label

train loader, val loader wraps these datasets into iterable folders with batch size of 64 

training data is shuffled for each epoch

In [6]:
train_ds = FERDataset(train_df, transform)
val_ds   = FERDataset(val_df, transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)


In [7]:
import torch
import torch.nn as nn
from torchvision import models

model = models.resnet18(weights="IMAGENET1K_V1")

# Replace FC layer
model.fc = nn.Linear(512, 5)

# Train only FC
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to("cuda" if torch.cuda.is_available() else "cpu")


# -----------------------------
# Loss & Optimizer
# -----------------------------
criterion = nn.CrossEntropyLoss()

# Adam optimizer with light weight decay for stability
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)





Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 192MB/s]


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total * 100

EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_acc = evaluate(train_loader)
    val_acc   = evaluate(val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.3f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
torch.save(model.state_dict(), "/kaggle/working/FER_resnet18_pretrained.pt")
print("Saved FER pretrained model.")

Epoch 1/15 | Loss: 326.476 | Train Acc: 79.78% | Val Acc: 72.13%
Epoch 2/15 | Loss: 209.324 | Train Acc: 89.40% | Val Acc: 71.95%
Epoch 3/15 | Loss: 117.560 | Train Acc: 95.62% | Val Acc: 72.51%
Epoch 4/15 | Loss: 46.908 | Train Acc: 97.44% | Val Acc: 72.17%
Epoch 5/15 | Loss: 25.054 | Train Acc: 98.41% | Val Acc: 71.09%
Epoch 6/15 | Loss: 20.019 | Train Acc: 98.44% | Val Acc: 70.75%
Epoch 7/15 | Loss: 22.722 | Train Acc: 96.63% | Val Acc: 70.23%
Epoch 8/15 | Loss: 27.768 | Train Acc: 97.63% | Val Acc: 72.25%
Epoch 9/15 | Loss: 18.820 | Train Acc: 99.25% | Val Acc: 72.54%
Epoch 10/15 | Loss: 12.717 | Train Acc: 99.22% | Val Acc: 73.07%
Epoch 11/15 | Loss: 10.110 | Train Acc: 98.40% | Val Acc: 72.47%
Epoch 12/15 | Loss: 21.598 | Train Acc: 98.53% | Val Acc: 72.95%
Epoch 13/15 | Loss: 19.489 | Train Acc: 99.06% | Val Acc: 72.54%
Epoch 14/15 | Loss: 10.439 | Train Acc: 99.49% | Val Acc: 73.10%
Epoch 15/15 | Loss: 9.292 | Train Acc: 98.82% | Val Acc: 71.42%
Saved FER pretrained model.
